# Calling D (dlang) shared libs from Python

The D programming language is attractive for making native extensions
for Python, for several reasons:

- Syntax features of D are excellent
- Very fast compilation speed

But the primary benefit for this use case is to avoid the C runtime 
problem (at least, I think it will?). The possibility of building a 
binary once, and then using that same binary on different versions of 
Python is very attractive. Currently with Cython, for example, the
extension module must be built on the target platform for each version
of Python that is supported.

The idea is similar (in the basic idea) to how the _milksnake_ project 
uses CFFI to wrap rust-produced binaries in Python.

## The Goal

Our goal is to run a *Python* program that uses CFFI to call a function inside a native extension written in D.
Now the Python file:

Remember that you must install CFFI:

```shell
$ pip install cffi
```

The Python file below will use the ABI mode of loading the dll:

```python
from cffi import FFI

ffi = FFI()
ffi.cdef("int foo();")
C = ffi.dlopen("blah.dll")
x = C.foo()
print(f"Called the dll, and the result was {x}")
```

Running the file must produce the following:

```
$ python blah.py
Called the dll, and the result was 123 
```


## Windows

On Windows it's a bit odd because you need some entrypoint code
inside the shared library. This is just boilerplate though. Focus
on the actual exported functions at the bottom:

```d
import core.sys.windows.windows;
import core.sys.windows.dll;
import core.stdc.stdio;

__gshared HINSTANCE g_hInst;

extern (Windows)
BOOL DllMain(HINSTANCE hInstance, ULONG ulReason, LPVOID pvReserved)
{
    switch (ulReason)
    {
	case DLL_PROCESS_ATTACH:
	    g_hInst = hInstance;
	    dll_process_attach( hInstance, true );
	    break;

	case DLL_PROCESS_DETACH:
	    dll_process_detach( hInstance, true );
	    break;

	case DLL_THREAD_ATTACH:
	    dll_thread_attach( true, true );
	    break;

	case DLL_THREAD_DETACH:
	    dll_thread_detach( true, true );
	    break;

        default:
    }
    return true;
}

extern (C) int foo() {
    return 123;
}

```

You will also need a `.def` file:

```
LIBRARY blah

CODE SHARED EXECUTE
DATA WRITE

EXPORTS
    foo
```

To build the `.dll`, run this:

```
$ dmd -m64 -betterC -shared -ofblah.dll blah.d blah.def
```

- On Windows you must supply `-m64` to make a 64-bit shared library. This
is important if you want to load it into a 64-bit Python application.
- We're passing the `-betterC` flag to indicate we don't need the D 
standard library.
- Obviously `-shared` is necessary to make a shared library.

After building, you can inspect the DLL:

```
$ ls -lah blah.*
-rw-r--r-- 1 caleb 197121  697 May  5 12:01 blah.d
-rw-r--r-- 1 caleb 197121   69 May  5 12:06 blah.def
-rwxr-xr-x 1 caleb 197121 401K May  5 12:06 blah.dll
-rw-r--r-- 1 caleb 197121  787 May  5 12:06 blah.exp
-rw-r--r-- 1 caleb 197121 1.7K May  5 11:59 blah.lib
-rw-r--r-- 1 caleb 197121 1.8K May  5 12:06 blah.obj
```

Running the Python program `blah.py` now produces the expected output.